# The Critics Critique ML model 1 

## 0. import libraries

In [15]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

## 1. import and slice dataset 

In [5]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [6]:
len(all_sites)

15551

In [7]:
all_test = all_sites[0:100]

In [8]:
all_test.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0


## 2.2 Classifier with Transformers nlptown/bert-base-multilingual-uncased-sentiment

In [5]:
# This let us see prob for every batch 1-5

'''from adaptnlp import EasySequenceClassifier

classifier = EasySequenceClassifier()

sentences = classifier.tag_text(
    text=text_splitted,
    model_name_or_path="nlptown/bert-base-multilingual-uncased-sentiment",
    padding='longest',
    mini_batch_size=4
)

print("Tag Score Outputs:\n")
for sentence in sentences:
    print(f"Fragment {sentences.index(sentence)}: {sentence.labels}")'''

'from adaptnlp import EasySequenceClassifier\n\nclassifier = EasySequenceClassifier()\n\nsentences = classifier.tag_text(\n    text=text_splitted,\n    model_name_or_path="nlptown/bert-base-multilingual-uncased-sentiment",\n    padding=\'longest\',\n    mini_batch_size=4\n)\n\nprint("Tag Score Outputs:\n")\nfor sentence in sentences:\n    print(f"Fragment {sentences.index(sentence)}: {sentence.labels}")'

### Sentiment proccesing function Bert base Multilingual Uncased 

In [10]:
def sentiment_analysis_ind(df):
    global_stars = []
    
    # Select model
    nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(nlp_model)
    model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

    # Classifier
    classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer
    )
    
    # Adapt review lenght to max tokens
    n = 1900
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]
    #print(len(review_splitted))
    
    # Classifying 
    for text in review_splitted:
            stars = (classifier(text))
            global_stars.append(stars)

    return global_stars
    

In [9]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 2000
    
    # Select model
    nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(nlp_model)
    model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

    # Classifier
    classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer
    )
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    
    for text in review_splitted:
        stars = (classifier(text))
        global_stars.append(stars)

    return global_stars


In [10]:
all_test_small = all_test[:15]

In [11]:
all_test_small['stars'] = all_test_small['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Sentiment Synthetic Score Function

In [12]:
def stars_mean_to_score(review):
    
    points = []

    for classifications in review:
        for classification in classifications:

            #Mean of fragment
            grade = int(classification['label'].split(' ')[0])
            points.append(grade)  
        score = round(np.mean(points), 2)

    return(score)   

In [13]:
all_test_small['stars_mean'] = all_test_small['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Precission Score

In [ ]:
'''from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall'''

## 3 Fine Tuning BERT model classifier

Train the entire architecture. Train some layers while freezing others, Freeze the entire architecture https://huggingface.co/transformers/training.html

### 3.1 Get labels

In [14]:
all_test_small.head()

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5,"[[{'label': '3 stars', 'score': 0.435262888669...",3.60
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2,"[[{'label': '3 stars', 'score': 0.421717613935...",3.20
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0,"[[{'label': '3 stars', 'score': 0.415306359529...",3.50
3,meristation,https://as.com/meristation/2014/10/06/analisis...,Salva Fernàndez,Titanfall: IMC Rising,None,"Acción, First Person Shooter",PC 360 XBO,Titanfall recibe el último contenido descargab...,7.0,"[[{'label': '3 stars', 'score': 0.483339041471...",3.25
4,meristation,https://as.com/meristation/2014/04/16/analisis...,None,Moebius: Empire Rising,None,Aventura Gráfica,PC OSX IPH IPD AND,"“Aquellos que no pueden recordar el pasado, es...",6.8,"[[{'label': '4 stars', 'score': 0.495593637228...",3.20


In [54]:
def score_labels(score):
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [55]:
all_test_small['score_label'] = all_test_small['score'].apply(lambda x: score_labels(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
all_test_small

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5,"[[{'label': '3 stars', 'score': 0.435262888669...",3.60,'4 starts'
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2,"[[{'label': '3 stars', 'score': 0.421717613935...",3.20,'4 starts'
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0,"[[{'label': '3 stars', 'score': 0.415306359529...",3.50,'4 starts'
3,meristation,https://as.com/meristation/2014/10/06/analisis...,Salva Fernàndez,Titanfall: IMC Rising,None,"Acción, First Person Shooter",PC 360 XBO,Titanfall recibe el último contenido descargab...,7.0,"[[{'label': '3 stars', 'score': 0.483339041471...",3.25,'4 starts'
4,meristation,https://as.com/meristation/2014/04/16/analisis...,None,Moebius: Empire Rising,None,Aventura Gráfica,PC OSX IPH IPD AND,"“Aquellos que no pueden recordar el pasado, es...",6.8,"[[{'label': '4 stars', 'score': 0.495593637228...",3.20,'3 starts'
5,Vandal,https://vandal.elespanol.com/analisis/pc/waste...,Alberto González,Wasteland 2 - Análisis,inXile Entertainment,RPG Occidental Tactical RPG,PC Xbox One PS4 Switch,"Hablar de Wasteland, es hablar de uno de los v...",8.5,"[[{'label': '4 stars', 'score': 0.492015153169...",4.00,'4 starts'
6,revogamers,https://www.revogamers.net/analisis-w/analisis...,Carlos Firás,Manifold Garden,None,None,Nintendo Switch,"En el último Nindies Direct se anunció, y se l...",7.0,"[[{'label': '4 stars', 'score': 0.415507256984...",4.00,'4 starts'
7,Vandal,https://vandal.elespanol.com/analisis/ps4/devi...,Carlos Leiva,Devil May Cry 5 - Análisis,Capcom,Acción Hack and Slash,PS4 PC Xbox One Xbox Series X PS5,Si quieres conocer todos los secretos y los me...,9.0,"[[{'label': '3 stars', 'score': 0.339096069335...",3.75,'4 starts'
8,Gamereactor,https://www.gamereactor.es/pirates-of-black-co...,Mika Sorvari,Pirates of Black Cove,Paradox Interactive,Estrategia,PC,"En Pirates of Black Cove, el jugador puede nav...",8.0,"[[{'label': '4 stars', 'score': 0.542094528675...",4.00,'4 starts'
9,meristation,https://as.com/meristation/2011/09/03/analisis...,Pablo González,Crimson Alliance,None,"Acción, Aventura",360,Poco importa que un juego tenga grandes gráfic...,7.5,"[[{'label': '2 stars', 'score': 0.345419675111...",3.57,'4 starts'


### 3.2 Split dataset

In [21]:
#model.train()

In [27]:
full_texts = all_test_small['text']

In [57]:
full_labels = all_test_small['score_label']

In [58]:
train_texts, val_texts, train_labels, val_labels = train_test_split(full_texts, full_labels, test_size=.2)

In [59]:
val_texts = val_texts.astype(str).tolist()
train_texts = train_texts.astype(str).tolist()

In [60]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [61]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [62]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

In [64]:
from transformers import Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

In [65]:
trainer.train()
#trainer.evaluate()

TypeError: new(): invalid data type 'str'